In [9]:
import subprocess
import os

import pandas as pd

pd.set_option('display.max_rows', 500)

import requests
from bs4 import BeautifulSoup

import json

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Data Understanding

 The sources used for collection of data are as follows-
* John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19.git
* RKI, webscrape (webscraping) https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
* REST API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com/

## 1) GITHUB csv Data

In [10]:
git_pull = subprocess.Popen( "/usr/bin/git pull" , 
                     cwd = os.path.dirname( '../data/raw/COVID-19/' ), 
                     shell = True, 
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()


print("Error : " + str(error)) 
print("out : " + str(out))

Error : b'The system cannot find the path specified.\r\n'
out : b''


In [11]:
git_data_location = '../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_gitraw = pd.read_csv(git_data_location)

In [12]:
pd_gitraw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38288,38304,38324,38398,38494,38520,38544,38572,38606,38641
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,9844,9967,10102,10255,10406,10553,10704,10860,11021,11185
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,45469,45773,46071,46364,46653,46938,47216,47488,47752,48007
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,1199,1215,1215,1215,1261,1261,1301,1301,1344,1344
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,2805,2876,2935,2965,2981,3033,3092,3217,3279,3335


## 2) Webscraping

In [13]:
RKI_page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [14]:
soup = BeautifulSoup(RKI_page.content, 'html.parser')

In [15]:
html_onlytable=soup.find('table')

In [16]:
all_rows=html_onlytable.find_all('tr') #Since only one table is present on the web-page.

In [17]:
final_data_list=[]

In [18]:
for pos,rows in enumerate(all_rows):
   
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #data element is denoted as td
    final_data_list.append(col_list)

In [19]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

In [20]:
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,45.254,+184,1.423,"12,9",1.867
3,Bayern,61.974,+269,2.328,"17,8",2.645
4,Berlin,12.269,+15,532,"14,2",226
5,Branden­burg,4.006,+1,65,"2,6",169
6,Bremen,2.135,+5,65,"9,5",58


## 3) REST API calls

In [21]:
## data request for Germany
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [22]:
json_object=json.loads(data.content) 

In [23]:
type(json_object)

dict

In [24]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'serverGens', 'geometryType', 'spatialReference', 'fields', 'features'])

In [25]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])

In [26]:
pd_full_list=pd.DataFrame(full_list)


In [27]:
pd_full_list

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death
0,1,01,Schleswig-Holstein,Land,2896712,15,4231,1600034400000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,146.062156,4.573731e+10,2.881496e+06,161
1,2,02,Hamburg,Freie und Hansestadt,1841179,6,6825,1600034400000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,370.686392,2.089396e+09,4.188002e+05,268
2,3,03,Niedersachsen,Land,7982448,9,17911,1600034400000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,224.379789,1.299836e+11,4.008988e+06,667
3,4,04,Bremen,Freie Hansestadt,682986,5,2135,1600034400000,04,4132268b-54de-4327-ac1e-760e915112f1,312.597916,1.119157e+09,3.357177e+05,58
4,5,05,Nordrhein-Westfalen,Land,17932651,10,62261,1600034400000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,347.193508,8.782936e+10,2.648673e+06,1829
5,6,06,Hessen,Land,6265809,7,16877,1600034400000,06,93277ac4-e8fc-48c7-8940-028dc2ed66af,269.350694,5.235913e+10,2.148244e+06,539
6,7,07,Rheinland-Pfalz,Land,4084844,11,9764,1600034400000,07,e9b4296f-9be2-4e53-9a58-ccf1396cb03d,239.029936,4.783877e+10,1.774430e+06,246
7,8,08,Baden-Württemberg,Land,11069533,1,45254,1600034400000,08,80394ddf-c6a4-4a6e-be8e-0259a81b22a9,408.815801,8.151732e+10,2.544320e+06,1867
8,9,09,Bayern,Freistaat,13076721,2,61974,1600034400000,09,1ff920f4-62cd-4a4f-b8c9-f042f2a3e00a,473.926147,1.634855e+11,3.898618e+06,2645
9,10,10,Saarland,Land,990509,12,3245,1600034400000,10,e3396a6f-8a30-4fdf-8df7-def77dd38bea,327.609340,6.060692e+09,5.626789e+05,175


In [28]:
pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')

In [29]:
pd_full_list.shape[0]

16

# API access via REST service, e.g. USA data 

example of a REST conform interface (accessable after mandatory registration )

www.smartable.ai

In [30]:
# US for full list
headers = {
    'Cache-Control': 'no-cache',
    'Subscription-Key': '28ee4219700f48718be78b057beb7eb4',
}

response = requests.get('https://api.smartable.ai/coronavirus/stats/US', headers=headers)
print(response)

ConnectionError: HTTPSConnectionPool(host='api.smartable.ai', port=443): Max retries exceeded with url: /coronavirus/stats/US (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000020D3C004D88>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [34]:
US_dict=json.loads(response.content) # imports string
with open('../data/raw/SMARTABLE/US_data.json', 'w') as outfile:
    json.dump(US_dict, outfile,indent=2)

NameError: name 'response' is not defined

In [35]:
print(json.dumps(US_dict,indent=2)) #string dump

NameError: name 'US_dict' is not defined

# Individual States US

In [36]:
US_dict['stats']['breakdowns'][0]

NameError: name 'US_dict' is not defined

In [37]:
full_list_US_country=[]
for pos,each_dict in enumerate (US_dict['stats']['breakdowns'][:]):
    flatten_dict=each_dict['location']
    flatten_dict.update(dict(list(US_dict['stats']['breakdowns'][pos].items())[1: 7]) 
    )
    full_list_US_country.append(flatten_dict)

NameError: name 'US_dict' is not defined

In [38]:
pd.DataFrame(full_list_US_country).to_csv('../data/raw/SMARTABLE/full_list_US_country.csv',sep=';',index=False)